In [1]:
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import load
from sklearn.metrics import classification_report

<Figure size 1000x600 with 0 Axes>

In [2]:
wesad = load.WESAD()
df = wesad.separate_and_feature_extract(sample_n=420000) #700 * 60 * 10 (10mins)

Using ready-made DataFrame


In [3]:
df.shape

(30, 59, 20)

In [4]:
y = df[:, 0, 0]  # 取得 'label'
X = df[:, :, 2:]  # 移除 'label','subject'

In [5]:
y=y-1

In [6]:
import numpy as np

X = np.array(X, dtype=np.float32)

y = np.array(y, dtype=np.int32)

print("X dtype:", X.dtype)  # 應該是 float32
print("y dtype:", y.dtype)  # 應該是 int32

X dtype: float32
y dtype: int32


In [7]:
# 分割訓練測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(filters=32, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [47]:
def build_cnn_lstm_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(filters=32, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.LSTM(64))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [48]:
input_shape = (X_train.shape[1], X_train.shape[2]) #(59,18)
num_classes = len(np.unique(y_train))

print(input_shape)
print(num_classes)

(59, 18)
2


In [49]:
# 初始化模型
model = build_cnn_model(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 30
batch_size = 16
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/30


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.4514 - loss: 2.0589 - val_accuracy: 0.3333 - val_loss: 1.8420
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5347 - loss: 1.3115 - val_accuracy: 0.8333 - val_loss: 0.5622
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5069 - loss: 0.7084 - val_accuracy: 0.6667 - val_loss: 0.5378
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6944 - loss: 0.6224 - val_accuracy: 0.8333 - val_loss: 0.6497
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5833 - loss: 0.6486 - val_accuracy: 0.5000 - val_loss: 0.8016
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6181 - loss: 0.6727 - val_accuracy: 0.6667 - val_loss: 0.5567
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6736 - loss: 0.6096 - val_accuracy: 0.8333 - val_loss: 0.4778
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5278 - loss: 0.6894 - val_accuracy: 0.6667 - val_loss: 0.4733
Epoch 9/30

In [50]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
              precision    recall  f1-score   support

           1       0.33      0.50      0.40         2
           2       0.67      0.50      0.57         4

    accuracy                           0.50         6
   macro avg       0.50      0.50      0.49         6
weighted avg       0.56      0.50      0.51         6



In [51]:
# 初始化模型
model = build_cnn_lstm_model(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 30
batch_size = 16
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/30


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 308ms/step - accuracy: 0.5694 - loss: 0.6894 - val_accuracy: 0.3333 - val_loss: 0.8357
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5069 - loss: 0.7397 - val_accuracy: 0.3333 - val_loss: 0.7811
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5278 - loss: 0.7014 - val_accuracy: 0.3333 - val_loss: 0.7324
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5486 - loss: 0.6759 - val_accuracy: 0.8333 - val_loss: 0.6576
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5208 - loss: 0.6812 - val_accuracy: 0.6667 - val_loss: 0.6334
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6806 - loss: 0.6633 - val_accuracy: 0.6667 - val_loss: 0.6693
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5764 - loss: 0.6504 - val_accuracy: 0.6667 - val_loss: 0.6734
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5972 - loss: 0.6342 - val_accuracy: 0.6667 - val_loss: 0.5761
Epoch 9/30

In [52]:
y_pred_prob = model.predict(X_test)

# 將機率轉換為 0 或 1（閾值設為 0.5）
y_pred_classes = (y_pred_prob > 0.5).astype(int)

y_pred_single = np.argmax(y_pred_classes, axis=1)

# 輸出分類報告
report = classification_report(y_test, y_pred_single, target_names=['1', '2'])
print(report)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
              precision    recall  f1-score   support

           1       0.50      0.50      0.50         2
           2       0.75      0.75      0.75         4

    accuracy                           0.67         6
   macro avg       0.62      0.62      0.62         6
weighted avg       0.67      0.67      0.67         6



In [53]:
# 選擇 LSTM 是因為 生理訊號屬於時間序列數據，LSTM 能夠記住過去的資訊，並利用這些資訊影響未來的預測。
# 此外，LSTM 透過 記憶單元與門機制，有效解決了傳統 RNN 中的 梯度消失問題，使模型能夠學習並捕捉長時間的依賴關係，從而提升壓力偵測的準確性。